## Assignment 2

For this assignment you'll be looking at 2017 data on immunizations from the CDC. Your datafile for this assignment is in [NISPUF17.csv](NISPUF17.csv). A data users guide for this, which you'll need to map the variables in the data to the questions being asked, is available at [NIS-PUF17-DUG.pdf](NIS-PUF17-DUG.pdf).

### Question 1
Write a function called `proportion_of_education` which returns the proportion of children in the dataset who had a mother with the education levels equal to less than high school (<12), high school (12), more than high school but not a college graduate (>12) and college degree.

*This function should return a dictionary in the form of (use the correct numbers, do not round numbers):* 
```
    {"less than high school":0.2,
    "high school":0.4,
    "more than high school but not college":0.2,
    "college":0.2}
```

In [15]:
import pandas as pd

def proportion_of_education():
    df = pd.read_csv('NISPUF17.csv')
    education_counts = df['EDUC1'].value_counts()
    total_children = len(df)
    
    prediction_counts = {
        'less than high school' : education_counts[1]/total_children,
        'high school' : education_counts[2]/total_children,
        'more than high school but not college': education_counts[3]/total_children,
        'college': education_counts[4]/total_children
    }

    return prediction_counts

print(proportion_of_education())

{'less than high school': 0.10202002459160373, 'high school': 0.172352011241876, 'more than high school but not college': 0.24588090637625154, 'college': 0.47974705779026877}


In [16]:
assert type(proportion_of_education())==type({}), "You must return a dictionary."
assert len(proportion_of_education()) == 4, "You have not returned a dictionary with four items in it."
assert "less than high school" in proportion_of_education().keys(), "You have not returned a dictionary with the correct keys."
assert "high school" in proportion_of_education().keys(), "You have not returned a dictionary with the correct keys."
assert "more than high school but not college" in proportion_of_education().keys(), "You have not returned a dictionary with the correct keys."
assert "college" in proportion_of_education().keys(), "You have not returned a dictionary with the correct keys."

### Question 2

Let's explore the relationship between being fed breastmilk as a child and getting a seasonal influenza vaccine from a healthcare provider. Return a tuple of the average number of influenza vaccines for those children we know received breastmilk as a child and those who know did not.

*This function should return a tuple in the form (use the correct numbers:*
```
(2.5, 0.1)
```

In [35]:
def average_influenza_doses():
    df = pd.read_csv('NISPUF17.csv')

    breastfeeding_group = df[df['CBF_01'] == 1]
    non_breastfeeding_group = df[df['CBF_01'] == 2]

    avg_influenza_breastfeeding = breastfeeding_group['P_NUMFLU'].mean()
    avg_influenza_non_breastfeeding = non_breastfeeding_group['P_NUMFLU'].mean()
    
    return (avg_influenza_breastfeeding, avg_influenza_non_breastfeeding)

print(average_influenza_doses())

(1.8799187420058687, 1.5963945918878317)


In [36]:
assert len(average_influenza_doses())==2, "Return two values in a tuple, the first for yes and the second for no."

### Question 3
It would be interesting to see if there is any evidence of a link between vaccine effectiveness and sex of the child. Calculate the ratio of the number of children who contracted chickenpox but were vaccinated against it (at least one varicella dose) versus those who were vaccinated but did not contract chicken pox. Return results by sex. 

*This function should return a dictionary in the form of (use the correct numbers):* 
```
    {"male":0.2,
    "female":0.4}
```

Note: To aid in verification, the `chickenpox_by_sex()['female']` value the autograder is looking for starts with the digits `0.0077`.

In [89]:
def chickenpox_by_sex():
    df = pd.read_csv('NISPUF17.csv')
    vaccine_df = df[df['P_NUMVRC'] > 0]
    
    male_df = vaccine_df[vaccine_df['SEX'] == 1]
    female_df = vaccine_df[vaccine_df['SEX'] == 2]

    male_vaccinated = male_df[male_df['HAD_CPOX'] == 1]
    male_not_vaccinated = male_df[male_df['HAD_CPOX'] == 2]

    female_vaccinated = female_df[female_df['HAD_CPOX'] == 1]
    female_not_vaccinated = female_df[female_df['HAD_CPOX'] == 2]
    
    sex_by_ratio = {'male' : male_vaccinated.shape[0]/male_not_vaccinated.shape[0],
                    'female': female_vaccinated.shape[0]/female_not_vaccinated.shape[0]}
    return sex_by_ratio

print(chickenpox_by_sex())

{'male': 0.009675583380762664, 'female': 0.0077918259335489565}


In [90]:
assert len(chickenpox_by_sex())==2, "Return a dictionary with two items, the first for males and the second for females."

### Question 4
A correlation is a statistical relationship between two variables. If we wanted to know if vaccines work, we might look at the correlation between the use of the vaccine and whether it results in prevention of the infection or disease [1]. In this question, you are to see if there is a correlation between having had the chicken pox and the number of chickenpox vaccine doses given (varicella).

Some notes on interpreting the answer. The `had_chickenpox_column` is either `1` (for yes) or `2` (for no), and the `num_chickenpox_vaccine_column` is the number of doses a child has been given of the varicella vaccine. A positive correlation (e.g., `corr > 0`) means that an increase in `had_chickenpox_column` (which means more no’s) would also increase the values of `num_chickenpox_vaccine_column` (which means more doses of vaccine). If there is a negative correlation (e.g., `corr < 0`), it indicates that having had chickenpox is related to an increase in the number of vaccine doses.

Also, `pval` is the probability that we observe a correlation between `had_chickenpox_column` and `num_chickenpox_vaccine_column` which is greater than or equal to a particular value occurred by chance. A small `pval` means that the observed correlation is highly unlikely to occur by chance. In this case, `pval` should be very small (will end in `e-18` indicating a very small number).

[1] This isn’t really the full picture, since we are not looking at when the dose was given. It’s possible that children had chickenpox and then their parents went to get them the vaccine. Does this dataset have the data we would need to investigate the timing of the dose?

In [48]:
import pandas as pd
import numpy as np
import scipy.stats as stats

def corr_chickenpox():
    df = pd.read_csv('NISPUF17.csv')
    filtered_df = df[(df['HAD_CPOX'] == 1) | (df['HAD_CPOX'] ==2)]
    filtered_df = filtered_df.dropna(subset=['P_NUMVRC'])

    corr, pval = stats.pearsonr(filtered_df['HAD_CPOX'], filtered_df['P_NUMVRC'])
    return corr

result = corr_chickenpox()
result

0.07044873460147867

In [49]:
assert -1<=corr_chickenpox()<=1, "You must return a float number between -1.0 and 1.0."

### Thank you
## Author

[Neelesh Chidar](https://www.linkedin.com/in/neeleshchidar37/)